In [1]:
import tensorflow as tf
import numpy as np
from numpy import linalg as la

C:\Users\weisja15\AppData\Local\Continuum\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.reset_default_graph()
tf.set_random_seed(25)

In [80]:
def bini(A, B, steps, e=1e-8):
    
    #Check Dimensions
    (m, n) = A.get_shape().as_list()
    #rn assuming that m is bigger than n, nn and p
    (nn, p) = B.get_shape().as_list()
    if n != nn: raise ValueError("incompatible dimensions")
    
    #pre-allocate output matrix
    C = tf.zeros([m,p])
    
    """
    This is the notation I use from Bini's 1980 paper

    |A1, A4|  |B1, B2|  =  |C1, C2|
    |A2, A5|  |B3, B4|     |C3, C4|
    |A3, A6|               |C5, C6|
    """
    
    #Base case
    if steps == 0 or m == 1 or n == 1 or p == 1:
        C = tf.matmul(A,B)
        return C
    
    #Static peeling
    if (3**steps > m) or (2**steps > n) or (2**steps > p):
        raise ValueError("Too many steps/ too small matricies for static peeling")
    
    if (m % 3**steps) != 0:
        print("Statics Peel 1")
        extra_rows = m % 3**steps
        
        #C[:m-extra_rows, :] = 
        Cmat = bini(A[:m-extra_rows, :], B, steps, e)
        #C[m-extra_rows:, :] =
        
        # need to expand dims if slice of A is a vector, and expand dims if it is
        A_slice = A[m-extra_rows:, :]
        (slice_len, _) = A_slice.get_shape().as_list()
        if slice_len == 1:
            #vector case
            Crow = tf.matmul(tf.expand_dims(A_slice, 0), B)
        
        else:
            # don't need to expand dims
            Crow = tf.matmul(A_slice, B)
        
        C = tf.concat([Cmat, Crow], 0)
        return C
    
    
    if (n % 2**steps) != 0:
        print("Statics Peel 2")
        extra_cols = n % (2**steps)
        
        Cmat = bini(A[:, :n-extra_cols], B[:n-extra_cols,:], steps, e)
        
        A_slice = A[:, n-extra_cols:]
        B_slice = B[n-extra_cols:, :]
        (_, slice_len) = A_slice.get_shape().as_list()
        
        if slice_len == 1:
            #vector case
            Ccol = tf.matmul(tf.expand_dims(A_slice, 1), tf.expand_dims(B_slice, 0))
        
        else: 
            Ccol = tf.matmul(A_slice, 1, B_slice)
        
        C = tf.add(Cmat, Ccol)
        return C
    
    
    if (p % 2**steps) != 0:
        print("Statics Peel 3")
        multiP = p//(2**steps) #multipler to find how large to make the bini matrix
        extra_cols = p % (2**steps)
        
        Cmat = bini(A, B[:, :p-extra_cols], steps, e)
        
        B_slice = B[:, p-extra_cols:]
        (_, slice_len) = B_slice.get_shape().as_list()
        
        if slice_len == 1:
            #vector case
            Ccol = tf.matmul(A, tf.expand_dims(B_slice, 1))
        else:
            Ccol = tf.matmul(A, B_slice)
        
        C = tf.concat([Cmat, Ccol], 1)
        return C
    
    """
    Dynamic peeling causes issues because the ideal epsilon value is determined by 
    the shape of the matrix and in dynamic peeling, the shape of the matrix
    is changed every recursive step which results in dimensions with a different 
    ideal epsilon value
    
    #Dynamic peeling
    if m % 3 == 1:
        C[:m-1, :] = bini(A[:m-1,:],B, steps, e)
        C[m-1,:] = A[m-1,:]@B
        return C
    if m % 3 == 2:
        C[:m-2, :] = bini(A[:m-2,:],B, steps, e)
        C[m-2:,:] = A[m-2:,:]@B
        return C
    if n % 2 == 1:
        C = bini(A[:, :n-1], B[:n-1,:], steps, e)
        C = C + np.outer(A[:,n-1],B[n-1,:])
        return C
    if p % 2 == 1:
        C[:, :p-1] = bini(A, B[:,:p-1], steps, e)
        C[:,p-1] = A@B[:,p-1]
        return C
    """
 

    # split up the matricies once rows of A are divisible by 3
    # and cols of A and rows and cols of are divisible by 2
    m2 = int(m/3) #first third of the rows of A
    m3 = m2*2     #second third of the rows of A
    n2 = int(n/2) #half of the cols of A
    p2 = int(p/2) #half of the cols of B
    #nn2 = int(nn/2) # half of the rows of B
    
    A1 = A[:m2, :n2]
    A2 = A[m2:m3, :n2]
    A3 = A[m3:, :n2]
    A4 = A[:m2, n2:]
    A5 = A[m2:m3, n2:]
    A6 = A[m3:, n2:]
    
    B1 = B[:n2, :p2]
    B2 = B[:n2, p2:]
    B3 = B[n2:, :p2]
    B4 = B[n2:, p2:]
    
    #bini(A, B, steps, e=0.1)
    # conquer
    
    # check if TF has a special fun for scalar mul
    M1  = bini(tf.add(A1, A5)                  , tf.add(tf.scalar_mul(e, B1), B4)     , steps-1, e) 
    M2  = bini(A5                              , tf.subtract(-B3, B4)                 , steps-1, e)
    M3  = bini(A1                              , B4                                   , steps-1, e)
    M4  = bini(tf.add(tf.scalar_mul(e,A4), A5) , tf.add(tf.scalar_mul(-e, B1), B3)    , steps-1, e)
    M5  = bini(tf.add(A1, tf.scalar_mul(e, A4)), tf.add(tf.scalar_mul(e, B2), B4)     , steps-1, e)
    M6  = bini(tf.add(A2, A6)                  , tf.add(B1, tf.scalar_mul(e, B4))     , steps-1, e)
    M7  = bini(A2                              , tf.subtract(-B1, B2)                 , steps-1, e) 
    M8  = bini(A6                              , B1                                   , steps-1, e)
    M9  = bini(tf.add(A2, tf.scalar_mul(e, A3)), tf.subtract(B2, tf.scalar_mul(e, B4)), steps-1, e)
    M10 = bini(tf.add(tf.scalar_mul(e, A3), A6), tf.add(B1, tf.scalar_mul(e, B3))     , steps-1, e)
    
    # nation building
    # gonna have to con cat these, cant use indexing to put this together
    C1 = tf.scalar_mul((1/e), tf.add(M1, tf.subtract(tf.add(M2, M4), M3)))    #C[:m2, :p2]
    C2 = tf.scalar_mul((1/e), tf.add(-M3, M5))                                #C[:m2, p2:]
    C3 = tf.add(M4, tf.subtract(M6, M10))                                     #C[m2:m3, :p2] error from bini paper -M10 from +M10
    C4 = tf.subtract(M1, tf.add(M5, M9))                                      #C[m2:m3, p2:] error from bini paper -M5 from +M5
    C5 = tf.scalar_mul((1/e), tf.add(-M8,M10))                                #C[m3:, :p2]
    C6 = tf.scalar_mul((1/e), ( tf.add(M6, tf.subtract(tf.add(M7, M9), M8)))) #C[m3:, p2:]
    
    
    # need to put all of the above pieces together
    C13 = tf.concat([C1, C3], 0)
    C135 = tf.concat([C13, C5], 0)
    C24 = tf.concat([C2, C4], 0)
    C246 = tf.concat([C24, C6], 0)
    C = tf.concat([C135, C246], 1)
    """
    t1 = [[1, 2, 3], [4, 5, 6]]
    t2 = [[7, 8, 9], [10, 11, 12]]
    tf.concat([t1, t2], 0)  # [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]]
    tf.concat([t1, t2], 1)  # [[1, 2, 3, 7, 8, 9], [4, 5, 6, 10, 11, 12]]
    
    This is the notation I use from Bini's 1980 paper

    |A1, A4|  |B1, B2|  =  |C1, C2|
    |A2, A5|  |B3, B4|     |C3, C4|
    |A3, A6|               |C5, C6|
    """
    
    return C

In [81]:
steps = 1

In [82]:
def calculate_e(steps):
    e = (2**-52)**(1/(1+steps))
    return e

In [83]:
with tf.Session() as sess:
    a = np.random.rand(10,4)
    w = np.identity(4)

    a = tf.constant(a, dtype=tf.float64)
    w = tf.constant(w)
    
    m = sess.run(tf.matmul(a,w))
    b = sess.run(bini(a,w,steps, calculate_e(steps)))
    
    print("A: \n", sess.run(a), '\n')
    print("matmul: \n", m)
    print("Bini: \n", b)
    print("\n m-s: \n", m-b, '\n')
    print("Bini Error: ", la.norm(b-m, 'fro')/la.norm(b))


Statics Peel 1


ValueError: Shape must be rank 2 but is rank 3 for 'MatMul_1822' (op: 'MatMul') with input shapes: [1,1,4], [4,4].

In [ ]:
#with tf.Session() as sess:
    a = np.random.rand(4,4)
    w = np.identity(4)

    a = tf.constant(a, dtype=tf.float64)
    w = tf.constant(w)
    
    a = sess.run(a)
    w = sess.run(w)
    
    s = strass(a,w, 2)

In [ ]:
s